In [1]:
import os
import base64
from pydantic import BaseModel
from openai import AzureOpenAI
from platform_migration.api_utils import *

In [2]:
azure = Azure()
openai_client = azure.create_openai_client()

In [3]:
grade = "Primary 6"
type_of_error = "Mixed up the numbers given in the math word problem which leads to a wrong answer. But if you follow the mixed up wrong numbers, the solution is correct."

In [30]:
class MWP(BaseModel):
    word_problem: str
    solution: str
MWP_sys_prompt = ""
MWP_user_prompt = f"Generate a math word problem on finding percentage increase/decrease for {grade} students and then generate a correct solution to solve it."
math_prob = azure.get_GPT_struct_response(MWP_sys_prompt, MWP_user_prompt, MWP, openai_client=openai_client)

In [31]:
math_prob

{'status': 0,
 'word_problem': 'Sarah had 250 stickers last year. This year, she has 300 stickers. What is the percentage increase in the number of stickers Sarah has from last year to this year?',
 'solution': 'First, find the increase in the number of stickers: \n\nIncrease = 300 - 250 = 50\n\nNext, use the formula for percentage increase:\n\nPercentage Increase = (Increase / Original Amount) × 100%\n= (50 / 250) × 100%\n= 0.2 × 100%\n= 20%\n\nSo, the percentage increase in the number of stickers is 20%.',
 'response_time': 3.0512051582336426}

In [10]:
class WorkedExample(BaseModel):
    worked_example: str
    error_explanation: str

In [32]:
sys_prompt = f"""
#Role and Objective#
You are an experienced Singaporean Math teacher who wants to create worked examples on the common mistakes made by students when they solve math word problems. The worked examples will be used by students to understand these errors and to avoid them in their own work. 
Your task is to create a worked example with the following given information:
-	A math word problem
-	The correct solution to the math word problem
-	Grade of students the worked example should target
-	Type of error the worked example should be designed for

#Instructions#
- Output the worked example only.
- DO NOT include the reasoning process in the worked example.
- DO NOT include the math word problem in the worked example. 
- The final answer generated by the worked exmaple should be wrong.
- The worked example should be easy to understand for {grade} students.

#Output format#
Response in JSON format with the following fields below:
- worked_example: the generated worked example
- error_explanation: explanation of the error in the worked example
"""

user_prompt = f"""
The math word problem is: {math_prob['word_problem']}
The correct solution to the math word problem is:  {math_prob['solution']}
The grade of students the worked example should target is: {grade}
The type of error the worked example should be designed for is: {type_of_error}
"""

In [33]:
azure.get_GPT_struct_response(sys_prompt, user_prompt, WorkedExample, openai_client = openai_client)

{'status': 0,
 'worked_example': 'Step 1: Find the increase in the number of stickers.\nIncrease = 300 - 250 = 50\n\nStep 2: Use the formula for percentage increase.\nPercentage Increase = (Increase / New Amount) × 100%\n= (50 / 300) × 100%\n= 0.1667 × 100%\n= 16.67%\n\nSo, the percentage increase in the number of stickers is 16.67%.',
 'error_explanation': 'The mistake here was using the new amount (300 stickers) as the denominator when applying the percentage increase formula. The correct formula should use the original amount (250 stickers) as the denominator. Mixing up which amount to use is a common error and leads to the wrong percentage increase.',
 'response_time': 3.2332875728607178}